In [16]:
from enum import Enum
from typing import List, Union


class LogicOperator(Enum):
    """
    The set of accepted logic operators,
    the first value is there priority, like pemdas
    the second value is there operand count binary, unary, etc.
    """
    OR = 1,2
    AND = 2,2
    NOT = 3,1
    LPAREN = 4,0
    RPAREN = 5,0

class LogicTerminal(Enum):
    U = -1 # Unknown
    F = 0 # False
    T = 1 # True

class LogicExpression:
    def  __init__(self,expression: List[Union[LogicOperator, LogicTerminal, str]] = None):
        self.variables = {}
        self.expression = []
        if expression:
            self.set_expression(expression)

    def __str__(self):
        parts = []
        for value in self.expression:
            if isinstance(value, str): parts.append(value)
            else: parts.append(value.name)
        return " ".join(parts)

    def __iter__(self):
        """
        Iterates through expression, de aliasing all variables that are set true or false, leaving unknowns
        """
        return iter(self.expression)

    def set_expression(self, expression: List[Union[LogicOperator, LogicTerminal, str]]):
        """
        :param expression: Takes a list as an input, the list is an ordered sequence of 3 possible types:
        --Logic Operators using the LogicOperator Enumerator
        --Logic Terminals using the LogicTerminal enumerator
        --Strings, these can be any value and represent variables within the logic.
        This value is set and stored.
        """
        for value in expression:
            if isinstance(value, str):
                self.variables[value] = LogicTerminal.U

        self.expression = expression

    def set_variable(self, variable, value) -> bool:
        if variable not in self.variables:
            return False
        self.variables[variable] = value
        return True

    def get_variable(self, variable):
        return self.variables[variable]


class Agent:
    def __init__(self):
        pass

    def convert_postfix(self,expression: LogicExpression)-> LogicExpression:
        """
        Postfix conversion algorithm.
        Note that unknown variables are left as there variable name, not value
        :param expression: Takes a proper infix expression as input
        :return: Postfix expression
        """

        # Useful constants for conversion, update if adding more operators
        LEFT_ASSOCIATIVE = [
            LogicOperator.AND,
            LogicOperator.OR,
        ]
        RIGHT_ASSOCIATIVE = [
            LogicOperator.NOT
        ]


        output = []
        operator_stack = []


        for piece in expression:
            if isinstance(piece, LogicTerminal) or isinstance(piece, str):
                output.append(piece)

            elif piece is LogicOperator.LPAREN:
                operator_stack.append(piece)

            elif piece is LogicOperator.RPAREN:
                while operator_stack and operator_stack[-1] is not LogicOperator.LPAREN:
                    output.append(operator_stack.pop())
                if not operator_stack:
                    raise ValueError(f"Mismatched Parenthesis in expression:\n{expression}")
                operator_stack.pop()
            else:
                while operator_stack and operator_stack[-1] is not LogicOperator.LPAREN:
                    stack_top_value = operator_stack[-1].value[0]
                    piece_value = piece.value[0]

                    # Stacks operators on output by priority. When there are two of the same operator
                    # it's important to deal with them correctly
                    # Not and And have different associativity for example, treating them the same would result in errors in output.
                    if (stack_top_value > piece_value) or (stack_top_value == piece_value and piece in LEFT_ASSOCIATIVE):
                        output.append(operator_stack.pop())
                        continue
                    break
                operator_stack.append(piece)

        while operator_stack:
            operator = operator_stack.pop()
            if operator in (LogicOperator.LPAREN, LogicOperator.RPAREN):
                raise ValueError(f"Mismatched Parenthesis in expression:\n{expression}")
            output.append(operator)

        return LogicExpression(output)

    def apply_NOT(self,operands: List[LogicTerminal]):
        if operands[0] is LogicTerminal.F:
            return LogicTerminal.T
        elif operands[0] is LogicTerminal.T:
            return LogicTerminal.F
        else:
            return LogicTerminal.U

    def apply_AND(self,operands: List[LogicTerminal]):
        """
        Assumes operands are logic terminals, applies the and operation and returns a terminal
        """
        if LogicTerminal.F in operands:
            return LogicTerminal.F
        elif LogicTerminal.U in operands:
            return LogicTerminal.U
        else:
            return LogicTerminal.T

    def apply_OR(self,operands: List[LogicTerminal]):
        if LogicTerminal.T in operands:
            return LogicTerminal.T
        elif LogicTerminal.U in operands:
            return LogicTerminal.U
        else:
            return LogicTerminal.F


    def evaluate(self, expression: LogicExpression, convert_postfix: bool = True) -> LogicTerminal:
        """
        Requires a postfix expression for evaluation, if the expression is already in that format,
        then set convert_postfix to False
        :param expression:
        :param convert_postfix:
        :return:
        """
        if convert_postfix:
            postfix_expression = self.convert_postfix(expression)
        else:
            postfix_expression = expression

        evaluation_stack = []
        for piece in postfix_expression:
            if isinstance(piece, LogicTerminal):
                evaluation_stack.append(piece)
            elif isinstance(piece, str):
                variable_value = expression.get_variable(piece)
                evaluation_stack.append(variable_value)
            else: # Piece is an operator
                if len(evaluation_stack) < piece.value[1]: # Checks if there are enough operands for operation
                    raise ValueError(f"Invalid postfix Expression:\n{postfix_expression}")

                operands = [evaluation_stack.pop() for _ in range(piece.value[1])][::-1] # Creates a list of operands from the stack, retaining original order
                for operand in operands:
                    assert isinstance(operand, LogicTerminal), f"Invalid postfix Expression:\n{postfix_expression}"

                match piece:
                    case LogicOperator.NOT:
                        evaluation_stack.append(self.apply_NOT(operands))
                    case LogicOperator.AND:
                        evaluation_stack.append(self.apply_AND(operands))
                    case LogicOperator.OR:
                        evaluation_stack.append(self.apply_OR(operands))
                    case _:
                        raise ValueError(f"Invalid postfix expression:\n{postfix_expression}")

        assert len(evaluation_stack)==1,f"Invalid postfix expression:\n{postfix_expression}"
        return evaluation_stack.pop()



a = Agent()
# x AND (NOT y) OR T
expression = LogicExpression([
    "x",                          # variable
    LogicOperator.AND,            # AND
    LogicOperator.LPAREN,         # (
        LogicOperator.NOT,        # NOT
        "y",                      # variable
    LogicOperator.RPAREN,         # )
    LogicOperator.OR,             # OR
    LogicTerminal.T               # constant True
])

print(expression)
print(a.evaluate(expression))




x AND LPAREN NOT y RPAREN OR T
LogicTerminal.T
